# Training SDSS Color Models

This notebook demonstrates how to load SDSS fluxes from PhotoObj files via the BLISS pipeline and fit gaussian mixture model color priors for both stars and galaxies.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from os import environ
import torch
from pathlib import Path
from astropy.io import fits
from hydra import initialize, compose

from bliss.surveys.sdss import column_to_tensor, SDSSDownloader

In [ ]:
environ["BLISS_HOME"] = str(Path().resolve().parents[1])
with initialize(config_path=".", version_base=None):
    cfg = compose("config")

# Retrieving raw ratios from disk via BLISS prior

We have SDSS flux information stored internally through PhotoObj FITS tables downloaded from the SDSS database. These are accessible by the SloanDigitalSkySurvey object. Note that these files are not stored within our repository. To correctly load ratios, specify a root sdss_dir directory that itself contains sub-directories specific to the desired run/camcol/field you would like to investigate (i.e. sdss_dir/run/camcol/field/photoObj_file(s)). 

In [ ]:
def compute_flux_ratio(obj_fluxes):
    """Query SDSS frames to get flux ratios in units of electron count.

    Args:
        obj_fluxes: tensor of electron counts for a particular SDSS object

    Returns:
        ratios (Tensor): Ratio of electron counts for each light source in field
        relative to r-band
    """
    ratios = torch.zeros(obj_fluxes.size())

    sdss_bands = ["u", "g", "r", "i", "z"]

    for i in range(len(sdss_bands)):
        # result: ratio of fluxes in nmgys
        ratios[i] = np.log(obj_fluxes[i] / obj_fluxes[2])

    return ratios

In [ ]:
from tqdm import tqdm

def load_color_distribution(sdss_fields, sdss_dir):
    star_flux_lst = []
    gal_flux_lst = []

    # load all star, galaxy fluxes relative to r-band required for sampling
    for run, camcol, field in tqdm(sdss_fields):

        # Set photoObj file path
        # NOTE: This is the necessary directory structure!
        sdss_path = Path(sdss_dir)
        field_dir = sdss_path / str(run) / str(camcol) / str(field)
        po_path = field_dir / f"photoObj-{run:06d}-{camcol:d}-{field:04d}.fits"

        if not po_path.exists():
            SDSSDownloader(run, camcol, field, str(sdss_path)).download_po()
        msg = (
            f"{po_path} does not exist. "
            + "Make sure data files are available for fields specified in config."
        )
        assert Path(po_path).exists(), msg
        po_fits = fits.getdata(po_path)

        # retrieve object-specific information for ratio computing
        objc_type = column_to_tensor(po_fits, "objc_type").numpy()
        thing_id = column_to_tensor(po_fits, "thing_id").numpy()

        # mask fluxes based on object identity & validity
        galaxy_bools = (objc_type == 3) & (thing_id != -1)
        star_bools = (objc_type == 6) & (thing_id != -1)
        star_fluxes = column_to_tensor(po_fits, "psfflux") * star_bools.reshape(-1, 1)
        gal_fluxes = column_to_tensor(po_fits, "cmodelflux") * galaxy_bools.reshape(-1, 1)
        fluxes = star_fluxes + gal_fluxes

        for obj, _ in enumerate(objc_type):
            if thing_id[obj] != -1 and torch.all(fluxes[obj] > 0):
                if objc_type[obj] == 6:
                    star_flux_lst.append(fluxes[obj])
                elif objc_type[obj] == 3:
                    gal_flux_lst.append(fluxes[obj])

    star_log_fluxes = torch.stack(star_flux_lst, dim=0).log()
    star_log_ratios = torch.zeros((star_log_fluxes.size(0), 4))
    for i in range(4):
        star_log_ratios[:, i] = star_log_fluxes[:, i + 1] - star_log_fluxes[:, i]

    gal_log_fluxes = torch.stack(gal_flux_lst, dim=0).log()
    gal_log_ratios = torch.zeros((gal_log_fluxes.size(0), 4))
    for i in range(4):
        gal_log_ratios[:, i] = gal_log_fluxes[:, i + 1] - gal_log_fluxes[:, i]

    return star_log_ratios, gal_log_ratios

Running load_color_distribution gives us our star/galaxy ratios read directly from SDSS PhotoObj tables that will be used to fit our color models. 

Note: These ratios are relative to the r-band.

In [ ]:
# load color distributions from sdss files
fields = (
    [(94, 1, f) for f in np.arange(12, 492, 10, dtype=int)] +
    [(125, 1, f) for f in np.arange(15, 575, 10, dtype=int)] + 
    [(752, 1, f) for f in np.arange(15, 695, 10, dtype=int)] +
    [(3900, 6, f) for f in np.arange(16, 606, 10, dtype=int)]
)
star_log_ratios, gal_log_ratios = load_color_distribution(fields, "/data/sdss")

In [ ]:
# filter rows with extreme entries, these are probably due to measurement noise
print(star_log_ratios.size())
star_log_ratios = star_log_ratios[torch.all(star_log_ratios.abs() < 3, dim=1)]
print(star_log_ratios.size())

In [ ]:
print(gal_log_ratios.shape)
gal_log_ratios = gal_log_ratios[torch.all(gal_log_ratios.abs() < 4, dim=1)]
print(gal_log_ratios.shape)

We store these ratios as Numpy arrays for convenience.

In [ ]:
# convert to numpy arrays
import numpy as np

star_log_ratios_np = np.array(star_log_ratios)
gal_log_ratios_np = np.array(gal_log_ratios)

# Fitting Star GMM

For assurance, we fit a series of models by cross-validation (k = 5) and determine the best model based on the average log-likelihood of the data given the model.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.mixture import GaussianMixture

# Tune these!
# ----------------
min_components = 1
max_components = 12
niter = 5
# ----------------

comps = np.linspace(min_components, max_components, niter, dtype=int)
params = {"n_components": comps}
gmm_log_likelihood_score = lambda estimator, X: -estimator.score(X) # minimize BIC
cv = GridSearchCV(GaussianMixture(), params, verbose=1, scoring=gmm_log_likelihood_score)
cv.fit(star_log_ratios_np)

We can plot the results of our cross-validation experiment. 25 Gaussian components appears to maximize the log-likelihood for the star color model.

In [ ]:
import matplotlib.pyplot as plt

scores = cv.cv_results_['mean_test_score']
plt.plot(comps, scores)
plt.xlabel('Number of components')
plt.ylabel('Log Likelihood')
plt.title('Cross validated results for fitting star color prior')

### Testing Star GMM

Rather than fit to 25 components, we will use 64 components for the BLISS prior - this is a generous number of coefficients but should be no worse than fitting to a smaller number of components. We can predict the labels for this model given the test set to investigate the subsequent assignments visually.

In [ ]:
# evaluate best-fitting star color prior on test set
star_train, star_test = train_test_split(star_log_ratios_np, test_size=0.2)
n_comps = 64  # BLISS standard
gm_star = GaussianMixture(n_components=n_comps).fit(star_train)
labels = gm_star.predict(star_test)

Let's visually inspect the mixture coefficients. We don't expect a very even distribution over each component. 

In [ ]:
gm_star.weights_

In [ ]:
# Creating figure
def plot_components(gmm, labels, k, data, leg, type):
    """Plot points belonging to top-k components"""

    plt.figure().clear()

    weights = np.copy(gmm.weights_)
    x = np.argpartition(weights, -k)[-k:]

    d = {}
    for i in x:
        d[f"comp #{i}"] = data[labels == i]

    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")
    
    # Creating plot
    for i, key in enumerate(d.keys()):
        l = d[key]
        ax.plot(l[:, 1], l[:, 2], l[:, 0], ".", label=key)
    # ax.margins(x=-0.45, y=-0.45, z=-0.25)
    # plt.axis([0, 1.5, 0, 15])
    if type == "star":
        plt.title("3-D visualization of star flux ratios relative to R and their assigned color")
    elif type == "gal":
        plt.title("3-D visualization of gal flux ratios relative to R and their assigned color")
    ax.set_xlabel('G-U log ratio')
    ax.set_ylabel('R-I log ratio')
    ax.set_zlabel('I-Z log ratio')

    if leg:
        ax.legend()
    
    # show plot
    plt.show()

In [ ]:
plot_components(gm_star, labels, n_comps, star_test, False, "star")

Several point clouds are layered on top of each other in three-dimensional space (plotting only G, I and Z bands). Note that the more dispersed point clouds have a very low mixture coefficient.

If we restrict the labeled points to only those belonging to the twelve components with the highest mixture coefficients, we start to weed through the overlaid points.

In [ ]:
plot_components(gm_star, labels, 10, star_test, True, type="star")

# Fitting Gal GMM

We can do the same for our galaxy color prior!

In [ ]:
# Tune these!
# ---------------------
min_components = 1
max_components = 12
niter = 6
# ---------------------

comps = np.linspace(min_components, max_components, niter, dtype=int)
params = {"n_components": comps, "reg_covar": np.array([1e-4])}
gmm_log_likelihood_score = lambda estimator, X: -estimator.score(X)
cv = GridSearchCV(GaussianMixture(), params, verbose=1, scoring=gmm_log_likelihood_score)  # uses estimator.score by default, which is log likelihood for GMM
cv.fit(gal_log_ratios_np)

In [ ]:
scores = cv.cv_results_['mean_test_score']
plt.plot(comps, scores)
plt.xlabel('Number of components')
plt.ylabel('Log likelihood')
plt.title('Cross validated results for fitting galaxy color prior')

Here, 45 components appears to maximize the log-likelihood metric.

### Testing Gal GMM

We can further test this model visually as before, using a standard 64 components.

In [ ]:
gal_train, gal_test = train_test_split(gal_log_ratios_np, test_size=0.2)
n_comps = 64
gm_gal = GaussianMixture(n_components=64).fit(gal_train)
labels = gm_gal.predict(gal_test)

In [ ]:
plot_components(gm_gal, labels, 10, gal_test, leg=True, type="gal")

This plot may appear surprising, but further inspection shows that the mixture coefficients corresponding to components #0 and #1 are 5 and 4 orders of magnitude larger than the rest, respectively.

### Saving best Gal/Star Color Models

Following training/evaluation, we save these models to disk via pickle to use for sampling color priors!

In [ ]:
import pickle

# save best-performing color models via pickle
star_file = "star_gmm_nmgy.pkl"
gal_file = "gal_gmm_nmgy.pkl"

with open(star_file, 'wb') as f:
    pickle.dump(gm_star, f)

with open(gal_file, 'wb') as f:
    pickle.dump(gm_gal, f)

In [ ]:
gm_star.sample()[0]